In [32]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

### Tables in the process

In [33]:
cols = 'trade name qty_x price reason_x qty_y buy_target sell_target reason_y'.split()
colt = 'trade name qty price active reason market'.split()

In [53]:
format_dict = {
    'qty':'{:,}','qty_x':'{:,}','qty_y':'{:,}','available_qty':'{:,}','eps_a':'{:.2f}','eps_b':'{:.2f}',
    'volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
    'pe':'{:.2f}','pbv':'{:.2f}','max':'{:.2f}','min':'{:.2f}','dividend':'{:.4f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}','daily_volume':'{:,.2f}','dly_vol':'{:,.2f}',
    'price':'{:.2f}','target_price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}','amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}','percent':'{:,.2f}',
    'q_amt':'{:,}','y_amt':'{:,}','aq_amt':'{:,}','ay_amt':'{:,}','yoy_gain':'{:,}','acc_gain':'{:,}',    
    'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
    'yoy_pct':'{:.2f}%','acc_pct':'{:.2f}%','yield':'{:.2f}%',
    'q_amt': '{:,}','y_amt': '{:,}','yoy_gain': '{:,}','q_amt_c': '{:,}','q_amt_p': '{:,}','aq_amt': '{:,}','ay_amt': '{:,}',
    'acc_gain': '{:,}','latest_amt': '{:,}','previous_amt': '{:,}','inc_amt': '{:,}',
    'inc_amt_pq': '{:,}','inc_amt_py': '{:,}',    'latest_amt_q': '{:,}','previous_amt_q': '{:,}','inc_amt_q': '{:,}',
    'latest_amt_y': '{:,}','previous_amt_y': '{:,}','inc_amt_y': '{:,}','kind_x': '{:,}',
    'inc_pct': '{:.2f}%','inc_pct_q': '{:.2f}%','pct': '{:.2f}%','percent': '{:.2f}%',
    'inc_pct_y': '{:.2f}%','inc_pct_pq': '{:.2f}%','inc_pct_py': '{:.2f}%','mean_pct': '{:.2f}%','std_pct': '{:.2f}%',      
}

### Begin of Loop

In [64]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.sort_values(['trade','name'],ascending=[True,True]).style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.00,"100,000.00",DOS,SET,1
1,B,KCE,"2,000",81.00,"162,000.00",RD07p,SET50,2
2,B,NOBLE,"21,000",5.85,"122,850.00",RD15%,SET,0
3,B,PTT,"3,000",36.50,"109,500.00",RD05%,SET50,0
4,B,RATCH,"3,000",42.75,"128,250.00",RD05%,SET50,1
5,B,RCL,"1,500",50.00,"75,000.00",ROUND,SET100,2
6,B,ROJNA,"18,000",6.30,"113,400.00",6L,SET,1
7,B,STA,"2,500",28.00,"70,000.00",RD15%,SET50,0
8,B,TCAP,"6,000",35.00,"210,000.00",1L,SET100,0
9,B,TISCO,"2,000",94.50,"189,000.00",CP1S,SET50,0


In [65]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlite)
stocks.shape

(54, 18)

In [58]:
df_merge = pd.merge(orders,stocks,on='name',how='inner')
df_merge.shape

(29, 25)

In [59]:
qty_mask = (df_merge.qty_x != df_merge.qty_y)
df_merge[qty_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
1,B,KCE,"2,000",81.00,RD07p,800,81.00,0.00,RD05pct


In [60]:
name = 'KCE'
sqlUpd = """
UPDATE stocks
SET qty = 800
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET qty = 800
WHERE name = 'KCE'



1

In [61]:
buy_price_mask = (df_merge.price != df_merge.buy_target) & (df_merge.trade == 'B')
df_merge[buy_price_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y


In [67]:
name = 'BCH'
sqlUpd = """
UPDATE stocks
SET buy_target = 21.3, sell_target = 23.4
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET buy_target = 21.3, sell_target = 23.4
WHERE name = 'BCH'



1

In [69]:
sell_price_mask = (df_merge.price != df_merge.sell_target) & (df_merge.trade == 'S')
df_merge[sell_price_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
15,S,GLOBAL,"2,500",21.50,5%,"2,500",19.00,21.00,5pct
16,S,IMH,"3,000",21.50,24H,"3,000",0.00,20.70,15pct
17,S,IP,"3,500",20.20,15pct,"3,500",19.20,23.20,15pct


In [70]:
name = 'BCH'
sqlUpd = """
UPDATE stocks
SET sell_target = 23.4
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET sell_target = 23.4
WHERE name = 'BCH'



1

In [71]:
reason_mask = (df_merge.reason_x != df_merge.reason_y)
df_merge[reason_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
1,B,KCE,"2,000",81.00,RD07p,800,81.00,0.00,RD05pct
5,B,RCL,"1,500",50.00,ROUND,"1,500",50.00,53.75,25%
10,B,TSTH,"45,000",1.42,RD05pct,"45,000",1.42,1.72,15pct
15,S,GLOBAL,"2,500",21.50,5%,"2,500",19.00,21.00,5pct
16,S,IMH,"3,000",21.50,24H,"3,000",0.00,20.70,15pct
18,S,IVL,"3,000",50.50,20%,"3,000",0.00,50.50,20pct
23,S,SAT,"2,500",25.25,24H,"2,500",0.00,25.25,15pct
25,S,SYNEX,"1,500",37.00,5%,"1,500",31.50,37.00,12H
27,S,TU,"6,000",21.80,5%,"6,000",0.00,21.80,5pct


In [72]:
name = 'SAT'
sqlUpd = """
UPDATE stocks
SET reason = '15pct'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET reason = '15pct'
WHERE name = 'SAT'



1

### End of Loop process

In [73]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

orders[colt].to_csv(output_file, header=True, index=False)
orders[colt].to_csv(data_file,   header=True, index=False)
orders[colt].to_csv(box_file,    header=True, index=False)

### After call ord-log

In [74]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [75]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

In [76]:
from itables import show
show(df[['trans','name','spd','reason','qty','target','current','chg','percent']])

trans,name,spd,reason,qty,target,current,chg,percent


In [89]:
sql = '''
UPDATE stocks
SET buy_target = 0
WHERE buy_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [90]:
sql = '''
UPDATE stocks
SET sell_target = 0
WHERE sell_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [91]:
sql = '''
UPDATE stocks
SET available_qty = 0
WHERE available_qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [92]:
sql = '''
UPDATE stocks
SET cost = 0
WHERE cost IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [93]:
sql = '''
UPDATE stocks
SET qty = 0
WHERE qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [94]:
qty_mask = (stocks.qty == 0) & (stocks.status.isin(['B','I','O','S']))
stocks[qty_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
9,529,MEGA,56.50,32.25,O,0.00,0.00,258.60,0.47,0.000000,0,-11,13,0,1,0,1L,SET100
10,547,JMT,70.75,34.19,O,0.00,0.00,770.01,1.17,40.500000,0,-9,14,0,1,12,3L,SET100
14,567,TQM,136.00,92.50,O,0.00,0.00,131.85,0.56,112.000000,0,-28,29,0,6,1,3L,SET100
31,650,STARK,5.45,3.68,O,0.00,0.00,267.73,0.68,0.000000,0,-7,8,0,0,0,6L,SET
35,666,BLA,44.25,23.00,O,0.00,0.00,594.49,0.59,0.000000,0,-9,12,0,0,0,6L,SET
40,674,TSE,3.70,2.34,O,0.00,0.00,17.47,0.71,0.000000,0,-6,7,0,0,0,6L,SET
52,692,AEONTS,0.00,0.00,O,0.00,0.00,0.00,0.00,0.000000,0,-4,4,0,0,0,,
53,693,SMT,0.00,0.00,O,0.00,0.00,0.00,0.00,0.000000,0,-4,4,0,0,0,,SET


In [95]:
buy_mask = (stocks.buy_target == 0) & (stocks.status.isin(['B','O']))
stocks[buy_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
9,529,MEGA,56.50,32.25,O,0.00,0.00,258.60,0.47,0.000000,0,-11,13,0,1,0,1L,SET100
10,547,JMT,70.75,34.19,O,0.00,0.00,770.01,1.17,40.500000,0,-9,14,0,1,12,3L,SET100
14,567,TQM,136.00,92.50,O,0.00,0.00,131.85,0.56,112.000000,0,-28,29,0,6,1,3L,SET100
31,650,STARK,5.45,3.68,O,0.00,0.00,267.73,0.68,0.000000,0,-7,8,0,0,0,6L,SET
35,666,BLA,44.25,23.00,O,0.00,0.00,594.49,0.59,0.000000,0,-9,12,0,0,0,6L,SET
40,674,TSE,3.70,2.34,O,0.00,0.00,17.47,0.71,0.000000,0,-6,7,0,0,0,6L,SET
52,692,AEONTS,0.00,0.00,O,0.00,0.00,0.00,0.00,0.000000,0,-4,4,0,0,0,,
53,693,SMT,0.00,0.00,O,0.00,0.00,0.00,0.00,0.000000,0,-4,4,0,0,0,,SET


In [86]:
name = 'EPG'
sqlUpd = """
UPDATE stocks
SET buy_target = 10.2
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET buy_target = 10.2
WHERE name = 'EPG'



1

In [87]:
sell_mask = (stocks.sell_target == 0) & (stocks.status.isin(['I','S']))
stocks[sell_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


In [88]:
name = 'WHART'
sqlUpd = """
UPDATE stocks
SET sell_target = 12.8
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET sell_target = 12.8
WHERE name = 'WHART'



1